In [19]:
import json
from pathlib import Path
import pickle
import numpy as np

In [3]:
split = {
  "train": 0.4,
  "val": 0.2,
  "test": 0.4,
}

In [34]:
num_data = 500

In [4]:
assert sum(split.values()) == 1, "split values must sum to 1.0"

In [5]:
def extract_info(data):
  text = data['task']['data']
  id = data['task']['id']
  found = False
  for result in data['result']:
    label_name = result['from_name']
    if label_name == 'indication':
      found = True
      label = result['value']['choices'][0]
      assert len(result['value']['choices']) == 1, f'more than one label selected for task {id}'
  if not found:
    print(f'no indication label found for task {id}')
    return None
  else:
    return {
      'id': id,
      'text': text,
      'label': label
    }

In [6]:
data_dir = Path('/gpfs/data/geraslab/ekr6072/projects/study_indication/data')
output_path = data_dir / 'dataset.pkl'

In [7]:
data_paths = list(data_dir.rglob('*.json'))

In [8]:
dataset = []
for data_path in data_paths:
  with open(data_path, 'r') as f: 
    task = json.load(f)
  data = extract_info(task)
  if data is not None:
    dataset.append(data)

no indication label found for task 3858
no indication label found for task 3814
no indication label found for task 3899
no indication label found for task 3871


In [27]:
ids = []
for data in dataset:
  id = data['id']
  ids.append(id)

In [31]:
sort_indices = np.argsort(ids)

In [37]:
train_max_id = int(split['train'] * num_data)
val_max_id = int(split['val'] * num_data) + train_max_id

In [40]:
train_ds = dataset[:train_max_id]
val_ds = dataset[train_max_id:val_max_id]
test_ds = dataset[val_max_id:]

In [41]:
dataset = {
  "train": train_ds,
  "val": val_ds,
  "test": test_ds
}

In [42]:
with open(output_path, 'wb') as f:
  pickle.dump(dataset, f)